Retrieve PDBs data from PDB and get allele information of HLA class 1 

In [24]:
import requests
import pandas as pd

In [217]:
df = pd.read_csv('../databases/mhc1_pdb/raw_mhc1_pdb_list.csv')

In [3]:
df.head()

,PDB_ID,Resolution,Release_date,MHC_Name,Species,Epitope,TCR_Bound,TCR_Name,TRAV_gene,TRBV_gene,Docking_angle_deg,Incident_angle_deg,Buried_surface_area_A**2,Shape_complementarity,Affinity_Mutations_ATLAS
0,5BS0,2.4,3/2/2016,HLA-A*01:01,Human,ESDPIVAQY,True,MAG-IC3,TRAV21,TRBV5-1,57.3,13.0,1991.0,0.67,0.0
1,1YDP,1.9,3/7/2005,HLA-G,NaN,RIIPRHLQL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2D31,3.2,3/13/2006,HLA-G,NaN,RIIPRHLQL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2DYP,2006-11-06,2.5,HLA-G,NaN,RIIPRHLQL,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3KYN,2.4,2/22/2010,HLA-G,NaN,KGPPAALTL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
def get_pdb_data_from_pdb(pdb_id):
    # Define the GraphQL query and variables
    query = '''
    query structure($id: String!) {
      entry(entry_id: $id) {
        rcsb_id
        entry {
          id
        }
    refine {
          pdbx_refine_id
          ls_d_res_high
          ls_R_factor_R_work
          ls_R_factor_R_free
          ls_R_factor_obs
      }
    em_3d_reconstruction {
          resolution
      }
      rcsb_accession_info {
          initial_release_date
      }
        polymer_entities {
          rcsb_polymer_entity_container_identifiers {
            entry_id
            entity_id
            asym_ids
            auth_asym_ids
            uniprot_ids
            reference_sequence_identifiers {
              database_accession
            }
          }
          rcsb_polymer_entity {
            pdbx_description
          }
          entity_poly {
            type
            rcsb_entity_polymer_type
            pdbx_seq_one_letter_code_can
            rcsb_sample_sequence_length
            rcsb_mutation_count
          }
          rcsb_entity_source_organism {
            scientific_name
            ncbi_scientific_name
            rcsb_gene_name {
              value
            }
          }
        }
      }
    }
    '''

    variables = {
        "id": pdb_id
    }

    endpoint_url = "https://data.rcsb.org/graphql"

    # Create a dictionary with the query and variables
    request_data = {
        "query": query,
        "variables": variables
    }

    response = requests.post(endpoint_url, json=request_data)

    # Check the response status
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        raise ConnectionError(f'Error: {response.status_code}\nServer response:\n{response.status_code}')
    
def get_mhc_chain_and_seq(data: dict) -> str:
    chain, seq = None, None
    for entity in data['data']['entry']['polymer_entities']:
        try:
            organism = entity['rcsb_entity_source_organism'][0]['scientific_name'].lower()
        except AttributeError: # no organism
            continue
        gene_list = entity['rcsb_entity_source_organism'][0]['rcsb_gene_name']
        if organism == 'homo sapiens':
            is_hla = False
            entity_description = entity['rcsb_polymer_entity']['pdbx_description'].lower()
            if 'hla' in entity_description: is_hla = True 
            if gene_list is not None:
                for gene in gene_list:
                    if gene['value'].upper().startswith('HLA'): is_hla = True          
            else:
                pass
            if is_hla:
                seq = entity['entity_poly']['pdbx_seq_one_letter_code_can']
                chain = entity['rcsb_polymer_entity_container_identifiers']['auth_asym_ids'][0]
                if len(chain) > 1: 
                    chain = chain[0]
                return chain, seq, organism
        elif organism == 'mus musculus':
            is_mhc = False
            entity_description = entity['rcsb_polymer_entity']['pdbx_description'].lower()
            if 'mhc' in entity_description: is_mhc = True 
            if 'histocompatibility' in entity_description: is_mhc = True 
            if gene_list is not None:
                for gene in gene_list:
                    if gene['value'].upper().startswith('H2-'): is_mhc = True   
            if is_mhc:
                seq = entity['entity_poly']['pdbx_seq_one_letter_code_can']
                chain = entity['rcsb_polymer_entity_container_identifiers']['auth_asym_ids'][0]
                if len(chain) > 1: 
                    chain = chain[0]
                return chain, seq, organism
        else:
            print(f'Unknown organism {organism}. Skipping')
            continue
            
    if chain is None or seq is None:
        raise NotImplementedError(f'Review {pdb_id} because no MHC data was found')

def get_resolution_from_pdb(data: dict) -> float:
    try: # X-ray
        return data['data']['entry']['refine'][0]['ls_d_res_high']
    except TypeError:
        try: # Cryo-EM
            return data['data']['entry']['em_3d_reconstruction'][0]['resolution']
        except TypeError: # NMR do not have resolution
            return None

def get_release_date(data: dict) -> str:
    return data['data']['entry']['rcsb_accession_info']['initial_release_date']

In [41]:
# Single case
#pdb_id = '2CIK'

data = get_pdb_data_from_pdb(pdb_id)
chain, seq, organism = get_mhc_chain_and_seq(data)
print(chain, seq, organism)

A GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTEPRAPWIEQEGPEYWDRNTQIFKTNTQTYRESLRNLRGYYNQSEAGSHIIQRMYGCDLGPDGRLLRGHDQSAYDGKDYIALNEDLSSWTAADTAAQITQRKWEAARVAEQRRAYLEGLCVEWLRRYLENGKETLQRADPPKTHVTHHPVSDHEATLRCWALGFYPAEITLTWQRDGEDQTQDTELVETRPAGDRTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWEP homo sapiens


In [29]:
data['data']['entry']['rcsb_accession_info']['initial_release_date']

'2006-10-25T00:00:00Z'

In [36]:
mhc_sequences = {}
mhc_chains = {}
pdb_resolution = {}
mhc_organism = {}
release_date = {}

If it complains with this error:
`
TypeError: 'NoneType' object is not subscriptable
`
Just run the cell again and it will work (don't know why)

In [49]:
pdb_to_skip = [
    '7BYD',  # Macaca mulatta MHC1
    '6L9K',  # Mouse MHC1 expressed in Human organism?
    '6L9L',  # Mouse MHC1 expressed in Human organism?
    '6L9M',  # Mouse MHC1 expressed in Human organism?
    '6L9N',  # Mouse MHC1 expressed in Human organism?
    '7JI2',  # Mus musculus bactrianus MHC1?
]
for i, row in df.iterrows():
    pdb_id = row['PDB_ID']
    #if pdb_id in mhc_sequences and pdb_id in mhc_chains: continue
    if pdb_id in release_date and pdb_id in mhc_organism: continue
    if pdb_id in pdb_to_skip: continue
    data = get_pdb_data_from_pdb(pdb_id)
    chain, seq, organism = get_mhc_chain_and_seq(data)
    resolution = get_resolution_from_pdb(data)
    r_date = get_release_date(data)
    
    mhc_chains[pdb_id] = chain
    mhc_sequences[pdb_id] = seq
    pdb_resolution[pdb_id] = resolution
    mhc_organism[pdb_id] = organism
    release_date[pdb_id] = r_date
    

Unknown organism human cytomegalovirus hcmv. Skipping
Unknown organism hepatitis c virus genotype 1b (isolate hc-j1). Skipping
Unknown organism hepatitis b virus. Skipping
Unknown organism human. Skipping
Unknown organism human t-cell lymphotropic virus type 1 (african isolate). Skipping
Unknown organism toxoplasma gondii. Skipping
Unknown organism epstein-barr virus. Skipping
Unknown organism severe acute respiratory syndrome coronavirus 2. Skipping
Unknown organism severe acute respiratory syndrome coronavirus 2. Skipping
Unknown organism influenza a virus. Skipping
Unknown organism mycobacterium tuberculosis. Skipping
Unknown organism lymphocytic choriomeningitis virus. Skipping
Unknown organism lymphocytic choriomeningitis virus. Skipping
Unknown organism plasmodium berghei. Skipping


In [212]:
import pickle
with open('../databases/mhc1_pdb/mhc1_pdb_sequences.pickle', 'wb') as f:
    pickle.dump(mhc_sequences, f)
with open('../databases/mhc1_pdb/mhc1_pdb_chains.pickle', 'wb') as f:
    pickle.dump(mhc_chains, f)
with open('../databases/mhc1_pdb/mhc1_pdb_resolution.pickle', 'wb') as f:
    pickle.dump(pdb_resolution, f)

Add MHC sequence and chain in df, then identify alleles

In [221]:
def get_sequence(pdb_id):
    if pdb_id in mhc_sequences:
        return mhc_sequences[pdb_id]
    else:
        return None
    
def get_chain(pdb_id):
    if pdb_id in mhc_chains:
        return mhc_chains[pdb_id]
    else:
        return None
    
def get_resolution(pdb_id):
    if pdb_id in pdb_resolution:
        return pdb_resolution[pdb_id]
    else:
        return None

def get_mhc_organism(pdb_id):
    if pdb_id in mhc_organism:
        return mhc_organism[pdb_id]
    else:
        return None
    
def get_release_date(pdb_id):
    if pdb_id in release_date:
        return release_date[pdb_id]
    else:
        return None

# Applying the function to each row
df['mhc_seq'] = df['PDB_ID'].apply(get_sequence)
df['mhc_chain'] = df['PDB_ID'].apply(get_chain)
df['Resolution'] = df['PDB_ID'].apply(get_resolution)
df['Species'] = df['PDB_ID'].apply(get_mhc_organism)
df['Release_date'] = df['PDB_ID'].apply(get_release_date)

In [51]:
df.head()

,PDB_ID,Resolution,Release_date,MHC_Name,Species,Epitope,TCR_Bound,TCR_Name,TRAV_gene,TRBV_gene,Docking_angle_deg,Incident_angle_deg,Buried_surface_area_A**2,Shape_complementarity,Affinity_Mutations_ATLAS,mhc_seq,mhc_chain
0,5BS0,2.4,2016-03-02T00:00:00Z,A*01:01,homo sapiens,ESDPIVAQY,True,MAG-IC3,TRAV21,TRBV5-1,57.3,13.0,1991.0,0.67,0.0,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKME...,A
1,1YDP,1.9,2005-03-08T00:00:00Z,None,homo sapiens,RIIPRHLQL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHSMRYFSAAVSRPGRGEPRFIAMGYVDDTQFVRFDSDSASPRMEP...,A
2,2D31,3.2,2006-03-14T00:00:00Z,None,homo sapiens,RIIPRHLQL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GSHSMRYFSAAVSRPGRGEPRFIAMGYVDDTQFVRFDSDSACPRME...,A
3,2DYP,2.5,2006-11-07T00:00:00Z,None,homo sapiens,RIIPRHLQL,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MGSHSMRYFSAAVSRPGRGEPRFIAMGYVDDTQFVRFDSDSASPRM...,A
4,3KYN,2.4,2010-02-23T00:00:00Z,None,homo sapiens,KGPPAALTL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSHSMRYFSAAVSRPGRGEPRFIAMGYVDDTQFVRFDSDSASPRME...,A


In [273]:
df.to_csv('../databases/mhc1_pdb/mhc1_pdb_list_with_mhc1_seq_chains.csv', index=False)

In [1]:
import tempfile
from seq2hla import get_most_freq_allele_from_seq

def get_allele_from_seq(seq):
    with tempfile.NamedTemporaryFile(mode='w') as f:
        f.write(f'>temp\n{seq}')
        f.seek(0)
        # Get allele
        highest_frequency_alleles, mean_frequencies = get_most_freq_allele_from_seq(f.name)
    if highest_frequency_alleles is None: return None
    return highest_frequency_alleles[0]


In [13]:
mhc_allele = {}


"""
# Not needed after last update
pdb_to_skip = [
    '1YDP', # HLA-G Allele
    '2D31', # HLA-G Allele
    '2DYP', # HLA-G Allele
    '3KYN', # HLA-G Allele
    '3KYO', # HLA-G Allele
    '6AEE', # HLA-G Allele
    '6K60', # HLA-G Allele
    '5IUE', # HLA-F Allele
    '5KNM', # HLA-F Allele
    '1MHE', # HLA-E Allele
    '3BZE', # HLA-E Allele
    '3BZF', # HLA-E Allele
    '6GGM', # HLA-E Allele
    '6GH1', # HLA-E Allele
    '6GH4', # HLA-E Allele
    '6GHN', # HLA-E Allele
    '6GL1', # HLA-E*01:03 Allele
    '7P49', # HLA-E*01:03 Allele
    '7P4B', # HLA-E*01:03 Allele
]
"""

for i, row in df.iterrows():
    pdb_id = row['PDB_ID']
    if pdb_id in mhc_allele: continue
    if pdb_id in pdb_to_skip: continue
    try: 
        most_freq_allele = get_allele_from_seq(row['mhc_seq'])
    except ValueError:
        raise ValueError(f'Error with {pdb_id}')
    if most_freq_allele is None:
        print(f'/!\ No allele found for {pdb_id} /!\\')
    else:
        mhc_allele[pdb_id] = most_freq_allele


Blast command executed successfully for /tmp/tmpcfzf5e7n.
No high identity matches found in the BLAST results.
/!\ No allele found for 4U1I /!\
Blast command executed successfully for /tmp/tmpoi3gbp84.
No high identity matches found in the BLAST results.
/!\ No allele found for 3WUW /!\
Blast command executed successfully for /tmp/tmpm8q88yrs.
No high identity matches found in the BLAST results.
/!\ No allele found for 3X11 /!\
Blast command executed successfully for /tmp/tmp8ddyekq_.
No high identity matches found in the BLAST results.
/!\ No allele found for 3X12 /!\
Blast command executed successfully for /tmp/tmpfjpoqfuq.
No high identity matches found in the BLAST results.
/!\ No allele found for 1A1M /!\
Blast command executed successfully for /tmp/tmp030d6pzb.
No high identity matches found in the BLAST results.
/!\ No allele found for 1A1O /!\
Blast command executed successfully for /tmp/tmpqov0on9z.
No high identity matches found in the BLAST results.
/!\ No allele found for 1

In [10]:
# Manually added alleles
mhc_allele['6O9B'] = 'HLA-A3*01'
mhc_allele['6O9C'] = 'HLA-A3*01'

In [18]:
def get_mhc_allele(pdb_id):
    if pdb_id in mhc_allele:
        return mhc_allele[pdb_id]
    else:
        return None

df['MHC_Name'] = df['PDB_ID'].apply(get_mhc_allele)

In [52]:
df.to_csv('../databases/mhc1_pdb/prep_mhc1_pdb_list.csv', index=False)

# Inspect data

In [53]:
df = pd.read_csv('../databases/mhc1_pdb/prep_mhc1_pdb_list.csv')

In [61]:
df.head()

,PDB_ID,Resolution,Release_date,MHC_Name,Species,Epitope,TCR_Bound,TCR_Name,TRAV_gene,TRBV_gene,Docking_angle_deg,Incident_angle_deg,Buried_surface_area_A**2,Shape_complementarity,Affinity_Mutations_ATLAS,mhc_seq,mhc_chain
0,5BS0,2.4,2016-03-02T00:00:00Z,A*01:01,homo sapiens,ESDPIVAQY,True,MAG-IC3,TRAV21,TRBV5-1,57.3,13.0,1991.0,0.67,0.0,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKME...,A
1,1YDP,1.9,2005-03-08T00:00:00Z,NaN,homo sapiens,RIIPRHLQL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SHSMRYFSAAVSRPGRGEPRFIAMGYVDDTQFVRFDSDSASPRMEP...,A
2,2D31,3.2,2006-03-14T00:00:00Z,NaN,homo sapiens,RIIPRHLQL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GSHSMRYFSAAVSRPGRGEPRFIAMGYVDDTQFVRFDSDSACPRME...,A
3,2DYP,2.5,2006-11-07T00:00:00Z,NaN,homo sapiens,RIIPRHLQL,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MGSHSMRYFSAAVSRPGRGEPRFIAMGYVDDTQFVRFDSDSASPRM...,A
4,3KYN,2.4,2010-02-23T00:00:00Z,NaN,homo sapiens,KGPPAALTL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MSHSMRYFSAAVSRPGRGEPRFIAMGYVDDTQFVRFDSDSASPRME...,A


In [55]:
df['MHC_Name'].value_counts()

MHC_Name
A*02:01      286
A*24:02       48
B*27:05       33
B*35:01       30
B*07:02       28
B*57:01       27
A*11:01       26
B*35:08       19
B*08:01       18
B*27:09       14
B*15:02       11
B*57:03        9
B*15:01        9
A*03:01        9
B*44:02        9
B*44:05        8
A*01:01        8
C*08:02        7
A*02:06        6
B*44:03        6
A*68:01        5
C*06:02        4
B*37:01        4
B*53:01        3
A*30:03        3
C*05:01        3
B*58:11        3
B*58:01        3
C*07:02        3
B*39:01        3
B*18:01        3
HLA-A3*01      2
C*14:02        2
B*51:01        2
B*14:02        2
B*27:03        2
B*57:06        2
B*42:01        2
B*81:01        2
C*08:01        1
B*41:03        1
B*42:02        1
B*46:01        1
B*52:01        1
B*40:01        1
B*40:02        1
B*41:04        1
B*35:05        1
B*27:06        1
B*27:04        1
B*35:03        1
A*30:01        1
A*29:02        1
A*24:50        1
A*02:03        1
A*02:07        1
Name: count, dtype: int64

In [63]:
# Group by 'MHC_Name' and aggregate the corresponding 'PDB_IDs' as lists
result_df = df.groupby('MHC_Name')['PDB_ID'].apply(list).reset_index()

# Get the value counts for 'MHC_Name'
value_counts = df['MHC_Name'].value_counts()

# Merge the value counts with the resulting DataFrame
result_df['Count'] = result_df['MHC_Name'].map(value_counts)

result_df.head()

,MHC_Name,PDB_ID,Count
0,A*01:01,"[5BS0, 5BRZ, 1W72, 3BO8, 4NQV, 4NQX, 6AT9, 6MPP]",8
1,A*02:01,"[7Q99, 7Q9A, 7Q9B, 1OGA, 7QPJ, 2BNQ, 5TEZ, 7PD...",286
2,A*02:03,[3OX8],1
3,A*02:06,"[6UK4, 6P64, 6UK2, 3OXR, 6UJO, 6UJQ]",6
4,A*02:07,[3OXS],1


In [58]:
result_df.to_csv('../../hla1_sequence_diversity/hla1_pdb_diversity_with_pdbs.csv', index=False)

 Get a representative PDB for each HLA group with (the one with best resolution)

In [66]:
idx = df.groupby('MHC_Name')['Resolution'].idxmin()

# Get the corresponding PDB_IDs using the obtained index
best_resolution_info = df.loc[idx, ['MHC_Name', 'PDB_ID', 'mhc_chain', 'Resolution']]
best_resolution_info.head()

,MHC_Name,PDB_ID,mhc_chain,Resolution
635,A*01:01,3BO8,A,1.80
636,A*02:01,3D25,A,1.30
688,A*02:03,3OX8,A,2.16
689,A*02:06,3OXR,A,1.70
690,A*02:07,3OXS,A,1.75


In [67]:
best_resolution_info.to_csv('../../hla1_sequence_diversity/hla1_pdb_diversity_with_best_resolution_pdb.csv', index=False)